# Tools

## 1.- Setup inicial

### 1.1- Instalar librerías 

In [1]:
# Instalar librerías
#!pip install openai
#!pip install tenacity
#!pip install python-dotenv
#!spacy download es_core_news_sm

### 1.2.- Cargar librerías

In [2]:
import os
import json
import openai
import datetime
from dotenv import load_dotenv
from typing import Callable, List
from tenacity import retry, wait_random_exponential, stop_after_attempt

### 1.3.- OpenAI API

In [3]:
# Cargar secretos y configuración desde el archivo .env
load_dotenv()

# Configurar la clave de la API de OpenAI
openai.api_key = os.getenv("OPENAI_API_KEY")
embedding_model = os.getenv("OPENAI_EMBEDDING_MODEL")
print("OpenAI API key: {}".format(openai.api_key[:5] + '...' + openai.api_key[-5:]))

# Nombres de los modelos
gpt35_model = os.getenv("OPENAI_GPT35_MODEL")
gpt35_16k_model = os.getenv("OPENAI_GPT35_16K_MODEL")
gpt4_model = os.getenv("OPENAI_GPT4_MODEL")
print("GPT-3.5-Turbo model: {}".format(gpt35_model))
print("GPT-3.5-Turbo-16k model: {}".format(gpt35_16k_model))
print("GPT-4 model: {}".format(gpt4_model))

OpenAI API key: sk-eO...Re5lw
GPT-3.5-Turbo model: gpt-3.5-turbo
GPT-3.5-Turbo-16k model: gpt-3.5-turbo-16k
GPT-4 model: gpt-4


### 1.4.- Importar clase Tools

In [4]:
from agents.tools import Tool, Parameter

### 1.5.- Definir funciones útiles

In [5]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(3))
def generate_text(prompt, model=gpt35_model, messages=[], max_tokens=200, temperature=1.0, top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0, stop=None):
    _messages = []
    _messages.extend(messages)
    _messages.append({"role": "user", "content": prompt})

    response = openai.ChatCompletion.create(
        model=model,
        messages=_messages,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
        stop=stop
    )
    return response["choices"][0]["message"]["content"]

In [6]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(3))
def generate_text_with_function_call(prompt, model=gpt35_16k_model, messages=[], functions=[], max_tokens=-1, temperature=1.0, top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0, stop=None, function_call='auto'):
    _messages = []
    _messages.extend(messages)
    _messages.append({"role": "user", "content": prompt})
    
    if max_tokens == -1:
        max_tokens = None

    response = openai.ChatCompletion.create(
        model=model,
        messages=_messages,
        functions=functions,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty,
        stop=stop,
        function_call=function_call
    )
    
    if "function_call" in response["choices"][0]["message"]:
        return response["choices"][0]["message"]["function_call"]
    else:
        return None

## 2.- Ejemplo Function Calling

### 2.1 Suma de números o concatenación de strings

#### 2.1.1 Definición de funciones

In [7]:
def sumar(a: int, b: int) -> int:
    return a + b

def concatenar_strings(s1: str, s2: str) -> str:
    return s1 + s2

#### 2.1.2 Instanciar funciones como Tools

In [8]:
sumar_tool = Tool(
    name="sumar",
    func=sumar,
    description="Esta función recibe dos números enteros como entrada y devuelve la suma de estos dos números.",
    arguments=[
        Parameter(name="Sumando 1", description="Primer número", type=int, required=True),
        Parameter(name="Sumando 2", description="Segundo número", type=int, required=True)
    ]
)

concatenar_tool = Tool(
    name="concatenar_strings",
    func=concatenar_strings,
    description="Esta función toma dos palabras como entrada y las combina en una sola cadena de texto, sin agregar espacios ni caracteres adicionales.",
    arguments=[
        Parameter(name="String 1", description="Primera cadena", type=str, required=True),
        Parameter(name="String 2", description="Segunda cadena", type=str, required=True)
    ]
)

#Definir lista de tools
tools1 = [sumar_tool, concatenar_tool]

#### 2.1.3 Ejemplos de uso

#### Sumar números

In [9]:
input1 = "Sumar los números 5 y 7"

##### Llamada al modelo

In [11]:
function_calling1 = generate_text_with_function_call(prompt = input1, model = gpt35_16k_model, functions = [sumar_tool.get_schema(), concatenar_tool.get_schema()])
print(function_calling1)

{
  "name": "sumar",
  "arguments": "{\n  \"Sumando 1\": 5,\n  \"Sumando 2\": 7\n}"
}


##### Se ejecuta la función

In [13]:
name1, arguments1 = function_calling1.values()
sum1, sum2 = json.loads(arguments1).values()
for tool in tools1:
    if name1 == tool.name:
        result = tool.execute(sum1, sum2)
        print(f'El resultado es: {result}')

El resultado es: 12


---------------------------------------------------------------------------------------------------------------------------------------------------------

#### Sumar strings

In [14]:
input2 = 'Une estas palabras: "hola" y "mundo"'

##### Llamada al modelo

In [15]:
function_calling2 = generate_text_with_function_call(prompt = input2, model = gpt35_16k_model, functions = [sumar_tool.get_schema(), concatenar_tool.get_schema()])
print(function_calling2)

{
  "name": "concatenar_strings",
  "arguments": "{\n\"String 1\": \"hola\",\n\"String 2\": \"mundo\"\n}"
}


##### Se ejecuta la función

In [16]:
name2, arguments2 = function_calling2.values()
arg1, arg2 = json.loads(arguments2).values()
for tool in tools1:
    if name2 == tool.name:
        result = tool.execute(arg1, arg2)
        print(f'El resultado es: {result}')

El resultado es: holamundo


## 3.- Ejemplo de Prompting para replicar Function Calling

### 3.1 Operaciones con números

#### 3.1.1 Definición de funciones

In [17]:
# Funciones
def sumar(a: int, b: int) -> int:
    return a + b

def multiplicar(a: int, b: int) -> int:
    return a * b

def restar(a: int, b: int) -> int:
    return a - b

def al_cuadrado(numero: int) -> int:
    return numero * numero

#### 2.2.2 Instanciar funciones como Tools

In [18]:
# Crear instancias de la clase Tool para cada función
sumar_tool = Tool(
    name="sumar",
    func=sumar,
    description="Sumar dos números",
    arguments=[
        Parameter(name="a", description="Primer número", type=int, required=True),
        Parameter(name="b", description="Segundo número", type=int, required=True)
    ]
)

multiplicar_tool = Tool(
    name="multiplicar",
    func=multiplicar,
    description="Multiplicar dos números",
    arguments=[
        Parameter(name="a", description="Primer número", type=int, required=True),
        Parameter(name="b", description="Segundo número", type=int, required=True)
    ]
)

restar_tool = Tool(
    name="restar",
    func=restar,
    description="Restar dos números",
    arguments=[
        Parameter(name="a", description="Primer número", type=int, required=True),
        Parameter(name="b", description="Segundo número", type=int, required=True)
    ]
)

cuadrado_tool = Tool(
    name="al_cuadrado",
    func=al_cuadrado,
    description="Calcular el cuadrado de un número",
    arguments=[
        Parameter(name="numero", description="Número para elevar al cuadrado", type=int, required=True)
    ]
)

# Definir lista de herramientas
tools2 = [sumar_tool, multiplicar_tool, restar_tool, cuadrado_tool]